In [28]:
import pandas as pd

In [29]:
folder = '.\\input_files\\'
file = '231352191_Mercy Catholic Medical Center of Southeastern Pennsylvania_standardcharges .xlsx'

In [30]:
df = pd.read_excel(folder + file)

In [31]:
# Find the index of the row that starts with 'Code'
code_index = df.index[df.iloc[:, 0] == 'Code'][0]

# Set the header to the values in the 'Code' row
df.columns = df.iloc[code_index]

# Drop the 'Code' row, which is now redundant with the header
df = df.drop(code_index)

# Reset the index to start from 0
df = df.reset_index(drop=True)

In [32]:
# Drop completely empty rows
df.dropna(how='all', inplace=True)


In [33]:
df.rename(columns={
    'Code': 'code',
    'Description': 'description',
    'Type': 'setting',
}, inplace=True)

In [35]:
df.drop('Derived contracted rate', axis=1, inplace=True)

In [36]:
cols = df.columns.tolist()
id_vars = cols[:3]
value_vars = cols[3:]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name='standard_charge')

In [37]:
df.dropna(subset='standard_charge', inplace=True)

In [38]:
df['payer'] = df['payer'].str.replace('__Derived Contracted Rate', '')

In [39]:
mapping = {
    'Gross Charge': 'gross',
    'Discounted Cash Price': 'cash',
    'De-identified min contracted rate': 'min',
    'De-identified max contracted rate': 'max'
}

df['payer_category'] = df['payer'].map(mapping).fillna('payer')

In [45]:
df['code'] = df['code'].astype(str)

In [ ]:
df['setting'] = df['setting'].str.lower()

In [46]:
df.loc[df['code'].str.len() == 1, 'ms_drg'] = df['code'].str.zfill(3)
df.loc[df['code'].str.len() == 3, 'ms_drg'] = df['code']
df.loc[df['code'].str.len() == 5, 'hcpcs_cpt'] = df['code']

In [51]:
ccns = {'38-3176536': '230029',
 '383175878': '230156',
 '383176225': '230069',
 '824757260': '230259',
 '383521763': '230002',
 '382589966': '230066',
 '231352191': '390156',
 '510064326': '070002',
 '232794121': '390204',
 '231913910': '390258'}

ein = file.split('_')[0]

df['hospital_id'] = ccns[ein]

In [53]:
df['description'] = df['description'].str.strip()

In [ ]:
df['ms_drg'].fillna('', inplace=True)
df['hcpcs_cpt'].fillna('', inplace=True)
df['code'].fillna('', inplace=True)